# Crawling Web Berita CNN Indonesia

## Apa itu Crawling?

Crawling merupakan proses search engine untuk menemukan konten atau sesuatu situs halaman yang ada. Dalam bahasa kerennya crawling atau web crawling merupakan proses dimana search engine mengirimkan bot atau robot yang disebut (crawler atau spider) yang digunakan untuk menemukan konten-konten yang ada.

Yang dimaksud konten yaitu bervariasi, mulai dari halaman website yang saya lakukan ini, kemudian gambar, video, dokumen, dan lain sebagainya. Seperti halnya laba-laba, datang ke sebuah jaring dan melihat beberapa halaman website, kemudian mengikuti link yang terdapat di halaman website tersebut untuk mencari URL yang baru.

Ketika ada pengguna yang mencari sebuah konten di search engine dengan keyword tertentu, search engine akan mencarinya di indeks dan menentukan konten mana yang paling sesuai untuk pengguna tersebut.

## Proses Crawling

### Tool atau Library

In [2]:
# Import Library yang dibutuhkan
import requests as req
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

* **Request** digunakan untuk mengambil html/http dari sebuah website.
* **BeautifulSoup** berfungsi untuk mengambil data dari html/xml.
* **Time** berfungsi untuk memberikan jeda ketika ingin berpindah halaman.
* **Pandas** digunakan untuk membuat dataframe agar mudah dibaca.

### Code Program

#### Fungsi Clean_text()

In [3]:
def clean_text(text):
    text = text.replace('\xa0', '')
    text = text.replace('ADVERTISEMENT', '')
    text = text.replace('SCROLL TO CONTINUE WITH CONTENT', '')
    text = text.replace('\n', '')
    return text.strip()

Fungsi untuk membersihkan text yang tidak diinginkan, atau mengganggu.

#### Fungsi scrape_news()

In [4]:
def scrape_news(soup):
    berita = {}
    texts = []
    
    berita["judul"] = soup.title.text
    
    text_list = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
    for text in text_list.find_all("p"):
        cleaned_text = clean_text(text.text)
        texts.append(cleaned_text)
        
    berita["isi"] = " ".join(texts)
    berita["tanggal"] = soup.find("div", class_="container !w-[1100px] overscroll-none").find_all("div")[1].find_all("div")[4].text
    berita["kategori"] = soup.find("a", attrs={"aria-label": "link description", "dtr-act": "kanal"}).text
    return berita

Pada fungsi ini berisikan proses pembedahan dan juga pengambilan data pada sebuah website. Mengambil data sesuai struktur HTML/web yang ingin diambil datanya.

#### Fungsi get_html()

In [5]:
def get_html(url):
    try:
        response = req.get(url).text
        return bs(response, "html5lib")
    
    except Exception as e:
        print(e)
        return ""

Fungsi get_html dengan parameter url digunakan untuk mengambil response atau isi html dari web. Untuk mengambil response tersebut dibutuhkan library request, dan juga BeautifulSoup untuk mendapatkan isi html.

#### Fungsi get_news()

In [6]:
def get_news(soup):
    container = soup.find("div", class_="container !w-[1100px] overscroll-none")
    news_list = container.find_all("article", class_="flex-grow")
    return news_list

Fungsi get_news berfungsi untuk mengambil semua berita yang ada pada web, yang kemudian didapat kumpulan url berita yang ada pada halaman web.

#### Main Crawling

In [7]:
def main(halaman=1):
	url = "https://www.cnnindonesia.com/indeks/2/"
	news = []
	count = 1 # - n

	for counter in range(halaman):
		page = url + str(count)    
		soup = get_html(page)
		news_list = get_news(soup)
		
		for item in news_list:
			news_url = item.find('a')['href']
			# print(news_url)
			soup_news = get_html(news_url)
			result = scrape_news(soup_news)
			news.append(result)
		
		count+=1
		time.sleep(1)
		
	return news

Menyiapkan link/base url web berita yang ingin dicrawling, terdapat beberapa fungsi yang dipanggil yang sudah dibuat sebelumnya untuk mengambil informasi atau berita pada halaman website. Dalam code tersebut terdapat beberapa tahapan seperti fungsi:
* get_html
* get_news
* scrape_news

#### Main

In [8]:
halaman = int(input('Crawling berapa Halaman? '))
news = main(halaman)

Menjalankan program yang sudah dibuat dengan input berapa halaman yang ingin diambil.

### Convert Dataframe

In [9]:
df = pd.DataFrame.from_dict(news)
df

,judul,isi,tanggal,kategori
0,FOTO: Vaksinasi Polio Buat Anak-anak Gaza Digeber,,"Minggu, 01 Sep 2024 18:10 WIB",Internasional
1,"Pengunjung Incar Sepeda Listrik di Transmart, ...",Pengunjung puas belanja di Transmart Kota Kasa...,"Minggu, 01 Sep 2024 17:55 WIB",Ekonomi
2,Sepiring New York di Jantung Jakarta,Sepotong steak klasik dengan cita rasa New Yor...,"Minggu, 01 Sep 2024 17:50 WIB",Gaya Hidup
3,CdM Paralimpiade: All Indonesian Final Badmint...,Chef de Mission (CdM) Kontingen Indonesia di P...,"Minggu, 01 Sep 2024 17:40 WIB",Olahraga
4,VIDEO: Ribuan Massa Israel Demo Tentang Pemeri...,Ribuan warga Israel berunjuk rasa menentang pe...,"Minggu, 01 Sep 2024 17:15 WIB",Internasional
5,"Leo/Bagas Juara Korea Open 2024, Julukan 'Leba...",Ganda putra Indonesia Leo Rolly Carnando/Bagas...,"Minggu, 01 Sep 2024 17:03 WIB",Olahraga
6,Puan Buka IAPF: RI-Afrika Punya Sejarah Panjan...,Ketua Dewan Perwakilan Rakyat Republik Indones...,"Minggu, 01 Sep 2024 17:02 WIB",Nasional
7,"Wow, LED TV Diskon Jutaan Rupiah di Transmart ...",Transmart Full Day Sale datang lagi hari ini (...,"Minggu, 01 Sep 2024 17:00 WIB",Ekonomi
8,Smart TV Laku Keras di Transmart Full Day Sale...,Produk TV pintar alias smart TV menjadi perang...,"Minggu, 01 Sep 2024 16:55 WIB",Ekonomi
9,Satu Keluarga di Bintan Kepri Dilantik Jadi An...,"Satu keluarga di Bintan,Kepulauan Riau (Kepri)...","Minggu, 01 Sep 2024 16:50 WIB",Nasional
